In [ ]:
import requests
import json

## NOTE: These examples won't work because the environments have been disabled in the API
## but you can see the plots in the ./example folder
# task_path = "example/other_example.json"

task_path = r"week_4/jamiu_pod/phillip.s-enterprise_wiki-1-expert-1753263019/task.json"

with open(task_path, 'r') as file:
    task_json = json.load(file)

API_BASE_URL= "https://tau-bench.turing.com/"

response = requests.post(
    f"{API_BASE_URL}/task_verification",
    json=task_json,
    headers={"Content-Type": "application/json"}
)
response.status_code

In [ ]:
response.json()

In [ ]:
from IPython.display import Image
from IPython.display import display
import base64

plot_base64 = response.json()['plot_base64']
plot_image = base64.b64decode(plot_base64)

display(Image(data=plot_image))